<a href="https://colab.research.google.com/github/prp20/tensorflow_learning/blob/main/NLP_Practice_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

tf.__version__

'2.9.2'

In [ ]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews", split=('train[:80%]', 'train[80%:]', 'test'), as_supervised=True)

In [ ]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
train_labels = np.asarray(train_labels).astype('float32').reshape((-1,1))
test_examples, test_labels = tfds.as_numpy(test_data)
test_labels = np.asarray(test_labels).astype('float32').reshape((-1,1))

In [ ]:
for val in range(10):
  print([train_examples[val].split()])

[[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie.', b"Don't", b'be', b'lured', b'in', b'by', b'Christopher', b'Walken', b'or', b'Michael', b'Ironside.', b'Both', b'are', b'great', b'actors,', b'but', b'this', b'must', b'simply', b'be', b'their', b'worst', b'role', b'in', b'history.', b'Even', b'their', b'great', b'acting', b'could', b'not', b'redeem', b'this', b"movie's", b'ridiculous', b'storyline.', b'This', b'movie', b'is', b'an', b'early', b'nineties', b'US', b'propaganda', b'piece.', b'The', b'most', b'pathetic', b'scenes', b'were', b'those', b'when', b'the', b'Columbian', b'rebels', b'were', b'making', b'their', b'cases', b'for', b'revolutions.', b'Maria', b'Conchita', b'Alonso', b'appeared', b'phony,', b'and', b'her', b'pseudo-love', b'affair', b'with', b'Walken', b'was', b'nothing', b'but', b'a', b'pathetic', b'emotional', b'plug', b'in', b'a', b'movie', b'that', b'was', b'devoid', b'of', b'any', b'real', b'meaning.', b'I', b'am', b'disappointed', b'that', b'there'

In [ ]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))

Training entries: 25000, test entries: 25000


In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
tv_layer = TextVectorization(max_tokens=10000, standardize="lower_and_strip_punctuation", split="whitespace", ngrams=None, output_mode="int", output_sequence_length=None, pad_to_max_tokens=True)

In [ ]:
output_seq_length = round(sum([len(i.split()) for i in train_examples])/len(train_examples))

In [ ]:
tv_layer = TextVectorization(max_tokens=10000, standardize="lower_and_strip_punctuation", split="whitespace", output_mode="int", output_sequence_length=output_seq_length, pad_to_max_tokens=True)

In [ ]:
tv_layer.adapt(train_examples)

In [ ]:
from tensorflow.keras import layers

embedding_layer = layers.Embedding(input_dim=10000, output_dim=128, input_length=output_seq_length)

## Model 0: Naive Bayes

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

In [ ]:
model_0.fit(train_examples, train_labels)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
#Evaluate the baseline model

model_0_score = model_0.score(test_examples, test_labels)
model_0_score

0.82956

In [ ]:
model_0_preds = model_0.predict(test_examples)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {
      "accuracy": model_accuracy,
      "precision":model_precision,
      "recall": model_recall,
      "f1_score": model_f1_score
  }
  return model_results

def return_callbacks(model_name):
  callbacks_list = []
  callbacks_list.append(tf.keras.callbacks.ModelCheckpoint("saved_models/"+model_name, monitor='val_loss', save_best_only='True', verbose=1))
  callbacks_list.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights='True'))
  return callbacks_list

In [ ]:
model_0_results = calculate_results(test_labels, model_0_preds)

In [ ]:
model_0_results

{'accuracy': 82.956,
 'precision': 0.8343036261758116,
 'recall': 0.82956,
 'f1_score': 0.828953229782028}

## Model 1: Simple Neural Network

In [ ]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = tv_layer(inputs)
x = embedding_layer(x)
x = layers.GlobalAveragePooling1D()(x)
output = layers.Dense(1, activation='sigmoid')(x)

model_1 = tf.keras.Model(inputs, output, name="nlp_model_1")
model_1.summary()

Model: "nlp_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 234)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 234, 128)          1280000   
                                                                 
 global_average_pooling1d_2   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non

In [ ]:
model_1.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

In [ ]:
history_1 = model_1.fit(train_examples, train_labels, epochs=20, validation_data=(test_examples, test_labels), callbacks=return_callbacks(model_1.name))

Epoch 1/20
775/782 [============================>.] - ETA: 0s - loss: 0.5360 - accuracy: 0.7744
Epoch 1: val_loss improved from inf to 0.39833, saving model to saved_models/nlp_model_1
782/782 [==============================] - 9s 11ms/step - loss: 0.5352 - accuracy: 0.7746 - val_loss: 0.3983 - val_accuracy: 0.8435
Epoch 2/20
782/782 [==============================] - ETA: 0s - loss: 0.3225 - accuracy: 0.8738
Epoch 2: val_loss improved from 0.39833 to 0.33239, saving model to saved_models/nlp_model_1
782/782 [==============================] - 8s 11ms/step - loss: 0.3225 - accuracy: 0.8738 - val_loss: 0.3324 - val_accuracy: 0.8592
Epoch 3/20
780/782 [============================>.] - ETA: 0s - loss: 0.2582 - accuracy: 0.9012
Epoch 3: val_loss improved from 0.33239 to 0.31347, saving model to saved_models/nlp_model_1
782/782 [==============================] - 8s 11ms/step - loss: 0.2582 - accuracy: 0.9012 - val_loss: 0.3135 - val_accuracy: 0.8717
Epoch 4/20
781/782 [=====================

In [ ]:
model_1.evaluate(test_examples, test_labels)
model_1_preds = model_1.predict(test_examples)

782/782 [==============================] - 3s 3ms/step


In [ ]:
tf.round(model_1_preds)

<tf.Tensor: shape=(25000, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)>

In [ ]:
test_labels

array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

In [ ]:
model_1_results = calculate_results(test_labels, tf.round(model_1_preds))

In [ ]:
model_1_results

{'accuracy': 87.1,
 'precision': 0.8710383006512077,
 'recall': 0.871,
 'f1_score': 0.8709966708884878}

## model 2: Model with 1 or 2 layers

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = tv_layer(inputs)
x = embedding_layer(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs, output, name="nlp_model_2")
model_2.summary()

Model: "nlp_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 234)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 234, 128)          1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 

In [ ]:
model_2.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
history_2 = model_2.fit(train_examples, train_labels, epochs=20, validation_data=(test_examples, test_labels),callbacks=return_callbacks(model_2.name))

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 0.0799 - accuracy: 0.9767
Epoch 1: val_loss improved from inf to 0.57213, saving model to saved_models/nlp_model_2
782/782 [==============================] - 9s 11ms/step - loss: 0.0799 - accuracy: 0.9767 - val_loss: 0.5721 - val_accuracy: 0.8404
Epoch 2/20
776/782 [============================>.] - ETA: 0s - loss: 0.0686 - accuracy: 0.9807
Epoch 2: val_loss did not improve from 0.57213
782/782 [==============================] - 8s 10ms/step - loss: 0.0685 - accuracy: 0.9807 - val_loss: 0.6531 - val_accuracy: 0.8359
Epoch 3/20
780/782 [============================>.] - ETA: 0s - loss: 0.0591 - accuracy: 0.9844
Epoch 3: val_loss did not improve from 0.57213
782/782 [==============================] - 10s 13ms/step - loss: 0.0590 - accuracy: 0.9844 - val_loss: 0.7415 - val_accuracy: 0.8368
Epoch 4/20
776/782 [============================>.] - ETA: 0s - loss: 0.0499 - accuracy: 0.9872
Epoch 4: val_loss did not improve fr

In [ ]:
model_2.evaluate(test_examples, test_labels)
model_2_preds = model_2.predict(test_examples)
model_2_results = calculate_results(test_labels, tf.round(model_2_preds))
model_2_results

782/782 [==============================] - 3s 4ms/step


{'accuracy': 84.04,
 'precision': 0.8404263626175211,
 'recall': 0.8404,
 'f1_score': 0.8403969100841793}

## Model 3: LSTM

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=10000, output_dim=128, embeddings_initializer="uniform", input_length=output_seq_length, name="embedding_2")

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = tv_layer(inputs)
x = model_3_embedding(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model_3 = tf.keras.Model(inputs, output, name="nlp_model_3")
model_3.summary()

Model: "nlp_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 234)              0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 234, 128)          1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                       

In [ ]:
model_3.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
history_3 = model_3.fit(train_examples, train_labels, epochs=20, validation_data=(test_examples, test_labels),callbacks=return_callbacks(model_3.name))

Epoch 1/20
781/782 [============================>.] - ETA: 0s - loss: 0.6749 - accuracy: 0.5553
Epoch 1: val_loss improved from inf to 0.70065, saving model to saved_models/nlp_model_3


782/782 [==============================] - 27s 29ms/step - loss: 0.6749 - accuracy: 0.5552 - val_loss: 0.7006 - val_accuracy: 0.5000
Epoch 2/20
781/782 [============================>.] - ETA: 0s - loss: 0.6814 - accuracy: 0.5517
Epoch 2: val_loss improved from 0.70065 to 0.68708, saving model to saved_models/nlp_model_3


782/782 [==============================] - 24s 31ms/step - loss: 0.6814 - accuracy: 0.5517 - val_loss: 0.6871 - val_accuracy: 0.5287
Epoch 3/20
781/782 [============================>.] - ETA: 0s - loss: 0.6059 - accuracy: 0.6724
Epoch 3: val_loss improved from 0.68708 to 0.58070, saving model to saved_models/nlp_model_3


782/782 [==============================] - 21s 27ms/step - loss: 0.6059 - accuracy: 0.6724 - val_loss: 0.5807 - val_accuracy: 0.7876
Epoch 4/20
779/782 [============================>.] - ETA: 0s - loss: 0.3438 - accuracy: 0.8619
Epoch 4: val_loss improved from 0.58070 to 0.34037, saving model to saved_models/nlp_model_3


782/782 [==============================] - 21s 26ms/step - loss: 0.3434 - accuracy: 0.8620 - val_loss: 0.3404 - val_accuracy: 0.8568
Epoch 5/20
782/782 [==============================] - ETA: 0s - loss: 0.2119 - accuracy: 0.9192
Epoch 5: val_loss did not improve from 0.34037
782/782 [==============================] - 21s 27ms/step - loss: 0.2119 - accuracy: 0.9192 - val_loss: 0.3408 - val_accuracy: 0.8558
Epoch 6/20
780/782 [============================>.] - ETA: 0s - loss: 0.1447 - accuracy: 0.9503
Epoch 6: val_loss did not improve from 0.34037
782/782 [==============================] - 16s 21ms/step - loss: 0.1446 - accuracy: 0.9504 - val_loss: 0.4001 - val_accuracy: 0.8517
Epoch 7/20
779/782 [============================>.] - ETA: 0s - loss: 0.0945 - accuracy: 0.9710
Epoch 7: val_loss did not improve from 0.34037
782/782 [==============================] - 16s 21ms/step - loss: 0.0947 - accuracy: 0.9710 - val_loss: 0.4741 - val_accuracy: 0.8442
Epoch 8/20
781/782 [===================

In [ ]:
model_3.evaluate(test_examples, test_labels)
model_3_preds = model_3.predict(test_examples)
model_3_results = calculate_results(test_labels, tf.round(model_3_preds))
model_3_results

782/782 [==============================] - 6s 7ms/step


{'accuracy': 85.68,
 'precision': 0.8568036536694135,
 'recall': 0.8568,
 'f1_score': 0.8567996334070617}

## Model 4: LSTM AND GRU

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
from tensorflow.keras import layers
model_4_embedding = layers.Embedding(input_dim=10000, output_dim=128, embeddings_initializer="uniform", input_length=output_seq_length, name="embedding_4")

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = tv_layer(inputs)
x = model_4_embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs, output, name="nlp_model_4")
model_4.summary()

Model: "nlp_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 234)              0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 234, 128)          1280000   
                                                                 
 lstm_1 (LSTM)               (None, 234, 64)           49408     
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                       

In [ ]:
model_4.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
history_4 = model_4.fit(train_examples, train_labels, epochs=20, validation_data=(test_examples, test_labels),callbacks=return_callbacks(model_4.name))

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 0.6854 - accuracy: 0.5507
Epoch 1: val_loss improved from inf to 0.68581, saving model to saved_models/nlp_model_4


782/782 [==============================] - 37s 44ms/step - loss: 0.6854 - accuracy: 0.5507 - val_loss: 0.6858 - val_accuracy: 0.5391
Epoch 2/20
780/782 [============================>.] - ETA: 0s - loss: 0.5160 - accuracy: 0.7227
Epoch 2: val_loss improved from 0.68581 to 0.36252, saving model to saved_models/nlp_model_4


782/782 [==============================] - 32s 41ms/step - loss: 0.5158 - accuracy: 0.7229 - val_loss: 0.3625 - val_accuracy: 0.8386
Epoch 3/20
781/782 [============================>.] - ETA: 0s - loss: 0.2495 - accuracy: 0.9014
Epoch 3: val_loss improved from 0.36252 to 0.32268, saving model to saved_models/nlp_model_4


782/782 [==============================] - 31s 39ms/step - loss: 0.2495 - accuracy: 0.9014 - val_loss: 0.3227 - val_accuracy: 0.8585
Epoch 4/20
781/782 [============================>.] - ETA: 0s - loss: 0.1618 - accuracy: 0.9402
Epoch 4: val_loss did not improve from 0.32268
782/782 [==============================] - 24s 31ms/step - loss: 0.1618 - accuracy: 0.9402 - val_loss: 0.3265 - val_accuracy: 0.8690
Epoch 5/20
780/782 [============================>.] - ETA: 0s - loss: 0.0917 - accuracy: 0.9701
Epoch 5: val_loss did not improve from 0.32268
782/782 [==============================] - 24s 31ms/step - loss: 0.0916 - accuracy: 0.9701 - val_loss: 0.4377 - val_accuracy: 0.8530
Epoch 6/20
780/782 [============================>.] - ETA: 0s - loss: 0.0517 - accuracy: 0.9846
Epoch 6: val_loss did not improve from 0.32268
782/782 [==============================] - 24s 31ms/step - loss: 0.0516 - accuracy: 0.9846 - val_loss: 0.5097 - val_accuracy: 0.8499
Epoch 7/20
782/782 [===================

In [ ]:
model_4.evaluate(test_examples, test_labels)
model_4_preds = model_4.predict(test_examples)
model_4_results = calculate_results(test_labels, tf.round(model_4_preds))
model_4_results

782/782 [==============================] - 9s 10ms/step


{'accuracy': 85.85199999999999,
 'precision': 0.8648228523777816,
 'recall': 0.85852,
 'f1_score': 0.8579062804011561}

## Model 5: BI-Directional RNN

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim=10000, output_dim=128, embeddings_initializer="uniform", input_length=output_seq_length, name="embedding_5")

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = tv_layer(inputs)
x = model_5_embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model_5 = tf.keras.Model(inputs, output, name="nlp_model_5")
model_5.summary()

Model: "nlp_model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 234)              0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 234, 128)          1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 234, 128)         98816     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                       

In [ ]:
model_5.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
history_5 = model_5.fit(train_examples, train_labels, epochs=20, validation_data=(test_examples, test_labels),callbacks=return_callbacks(model_5.name))

Epoch 1/20
781/782 [============================>.] - ETA: 0s - loss: 0.4589 - accuracy: 0.7774
Epoch 1: val_loss improved from inf to 0.36475, saving model to saved_models/nlp_model_5


782/782 [==============================] - 95s 114ms/step - loss: 0.4590 - accuracy: 0.7774 - val_loss: 0.3647 - val_accuracy: 0.8515
Epoch 2/20
782/782 [==============================] - ETA: 0s - loss: 0.2764 - accuracy: 0.8935
Epoch 2: val_loss improved from 0.36475 to 0.35377, saving model to saved_models/nlp_model_5


782/782 [==============================] - 61s 78ms/step - loss: 0.2764 - accuracy: 0.8935 - val_loss: 0.3538 - val_accuracy: 0.8528
Epoch 3/20
781/782 [============================>.] - ETA: 0s - loss: 0.2017 - accuracy: 0.9257
Epoch 3: val_loss improved from 0.35377 to 0.34702, saving model to saved_models/nlp_model_5


782/782 [==============================] - 62s 80ms/step - loss: 0.2018 - accuracy: 0.9257 - val_loss: 0.3470 - val_accuracy: 0.8527
Epoch 4/20
782/782 [==============================] - ETA: 0s - loss: 0.1399 - accuracy: 0.9507
Epoch 4: val_loss did not improve from 0.34702
782/782 [==============================] - 41s 53ms/step - loss: 0.1399 - accuracy: 0.9507 - val_loss: 0.3742 - val_accuracy: 0.8447
Epoch 5/20
781/782 [============================>.] - ETA: 0s - loss: 0.0977 - accuracy: 0.9663
Epoch 5: val_loss did not improve from 0.34702
782/782 [==============================] - 41s 53ms/step - loss: 0.0978 - accuracy: 0.9663 - val_loss: 0.4841 - val_accuracy: 0.8479
Epoch 6/20
781/782 [============================>.] - ETA: 0s - loss: 0.0710 - accuracy: 0.9770
Epoch 6: val_loss did not improve from 0.34702
782/782 [==============================] - 41s 52ms/step - loss: 0.0710 - accuracy: 0.9770 - val_loss: 0.5464 - val_accuracy: 0.8457
Epoch 7/20
782/782 [===================

In [ ]:
model_5.evaluate(test_examples, test_labels)
model_5_preds = model_5.predict(test_examples)
model_5_results = calculate_results(test_labels, tf.round(model_5_preds))
model_5_results

782/782 [==============================] - 15s 18ms/step


{'accuracy': 85.268,
 'precision': 0.8543144975626087,
 'recall': 0.85268,
 'f1_score': 0.8525099023842375}

## model 6: CNN

In [ ]:
tf.keras.backend.clear_session()
from tensorflow.keras import layers
model_6_embedding = layers.Embedding(input_dim=10000, output_dim=128, embeddings_initializer="uniform", input_length=output_seq_length, name="embedding_6")

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = tv_layer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model_6 = tf.keras.Model(inputs, output, name="nlp_model_6")
model_6.summary()

Model: "nlp_model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 234)              0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 234, 128)          1280000   
                                                                 
 conv1d (Conv1D)             (None, 230, 32)           20512     
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 64)                

In [ ]:
model_6.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
history_6 = model_6.fit(train_examples, train_labels, epochs=20, validation_data=(test_examples, test_labels),callbacks=return_callbacks(model_6.name))

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 0.3081 - accuracy: 0.8777
Epoch 1: val_loss improved from inf to 0.36352, saving model to saved_models/nlp_model_6


782/782 [==============================] - 15s 13ms/step - loss: 0.3081 - accuracy: 0.8777 - val_loss: 0.3635 - val_accuracy: 0.8405
Epoch 2/20
780/782 [============================>.] - ETA: 0s - loss: 0.1598 - accuracy: 0.9414
Epoch 2: val_loss did not improve from 0.36352
782/782 [==============================] - 9s 11ms/step - loss: 0.1597 - accuracy: 0.9415 - val_loss: 0.3890 - val_accuracy: 0.8484
Epoch 3/20
782/782 [==============================] - ETA: 0s - loss: 0.0587 - accuracy: 0.9829
Epoch 3: val_loss did not improve from 0.36352
782/782 [==============================] - 9s 11ms/step - loss: 0.0587 - accuracy: 0.9829 - val_loss: 0.5268 - val_accuracy: 0.8409
Epoch 4/20
780/782 [============================>.] - ETA: 0s - loss: 0.0132 - accuracy: 0.9975
Epoch 4: val_loss did not improve from 0.36352
782/782 [==============================] - 9s 12ms/step - loss: 0.0133 - accuracy: 0.9974 - val_loss: 0.6814 - val_accuracy: 0.8411
Epoch 5/20
777/782 [======================

In [ ]:
model_6.evaluate(test_examples, test_labels)
model_6_preds = model_6.predict(test_examples)
model_6_results = calculate_results(test_labels, tf.round(model_6_preds))
model_6_results

782/782 [==============================] - 3s 4ms/step


{'accuracy': 84.052,
 'precision': 0.8459849929470595,
 'recall': 0.84052,
 'f1_score': 0.8398877381972287}

In [ ]:
df = pd.DataFrame([model_0_results, model_1_results, model_2_results, model_3_results, model_4_results, model_5_results, model_6_results])

In [ ]:
df

,accuracy,precision,recall,f1_score
0,82.956,0.834304,0.82956,0.828953
1,87.100,0.871038,0.87100,0.870997
2,84.040,0.840426,0.84040,0.840397
3,85.680,0.856804,0.85680,0.856800
4,85.852,0.864823,0.85852,0.857906
5,85.268,0.854314,0.85268,0.852510
6,84.052,0.845985,0.84052,0.839888


## Model 7: pre-trained Layer

In [ ]:
tf.keras.backend.clear_session()
from tensorflow.keras import layers
import tensorflow_hub as hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE")

model_7 = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_7_USE")

model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
model_7.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
history_7 = model_7.fit(train_examples, train_labels, epochs=20, validation_data=(test_examples, test_labels),callbacks=return_callbacks(model_7.name))

Epoch 1/20
781/782 [============================>.] - ETA: 0s - loss: 0.3734 - accuracy: 0.8406
Epoch 1: val_loss improved from inf to 0.32506, saving model to saved_models/model_7_USE


782/782 [==============================] - 77s 92ms/step - loss: 0.3733 - accuracy: 0.8407 - val_loss: 0.3251 - val_accuracy: 0.8570
Epoch 2/20
781/782 [============================>.] - ETA: 0s - loss: 0.3245 - accuracy: 0.8587
Epoch 2: val_loss improved from 0.32506 to 0.32488, saving model to saved_models/model_7_USE


782/782 [==============================] - 67s 85ms/step - loss: 0.3245 - accuracy: 0.8586 - val_loss: 0.3249 - val_accuracy: 0.8568
Epoch 3/20
780/782 [============================>.] - ETA: 0s - loss: 0.3201 - accuracy: 0.8627
Epoch 3: val_loss improved from 0.32488 to 0.32327, saving model to saved_models/model_7_USE


782/782 [==============================] - 62s 80ms/step - loss: 0.3205 - accuracy: 0.8625 - val_loss: 0.3233 - val_accuracy: 0.8564
Epoch 4/20
782/782 [==============================] - ETA: 0s - loss: 0.3155 - accuracy: 0.8647
Epoch 4: val_loss improved from 0.32327 to 0.32241, saving model to saved_models/model_7_USE


782/782 [==============================] - 63s 80ms/step - loss: 0.3155 - accuracy: 0.8647 - val_loss: 0.3224 - val_accuracy: 0.8586
Epoch 5/20
780/782 [============================>.] - ETA: 0s - loss: 0.3110 - accuracy: 0.8674
Epoch 5: val_loss did not improve from 0.32241
782/782 [==============================] - 55s 70ms/step - loss: 0.3108 - accuracy: 0.8676 - val_loss: 0.3252 - val_accuracy: 0.8554
Epoch 6/20
781/782 [============================>.] - ETA: 0s - loss: 0.3065 - accuracy: 0.8675
Epoch 6: val_loss improved from 0.32241 to 0.32092, saving model to saved_models/model_7_USE


782/782 [==============================] - 64s 82ms/step - loss: 0.3064 - accuracy: 0.8676 - val_loss: 0.3209 - val_accuracy: 0.8577
Epoch 7/20
781/782 [============================>.] - ETA: 0s - loss: 0.3011 - accuracy: 0.8740
Epoch 7: val_loss improved from 0.32092 to 0.32076, saving model to saved_models/model_7_USE


782/782 [==============================] - 63s 81ms/step - loss: 0.3011 - accuracy: 0.8739 - val_loss: 0.3208 - val_accuracy: 0.8585
Epoch 8/20
781/782 [============================>.] - ETA: 0s - loss: 0.2969 - accuracy: 0.8749
Epoch 8: val_loss improved from 0.32076 to 0.31732, saving model to saved_models/model_7_USE


782/782 [==============================] - 65s 84ms/step - loss: 0.2968 - accuracy: 0.8749 - val_loss: 0.3173 - val_accuracy: 0.8600
Epoch 9/20
781/782 [============================>.] - ETA: 0s - loss: 0.2897 - accuracy: 0.8782
Epoch 9: val_loss did not improve from 0.31732
782/782 [==============================] - 53s 68ms/step - loss: 0.2900 - accuracy: 0.8782 - val_loss: 0.3237 - val_accuracy: 0.8572
Epoch 10/20
782/782 [==============================] - ETA: 0s - loss: 0.2841 - accuracy: 0.8836
Epoch 10: val_loss did not improve from 0.31732
782/782 [==============================] - 54s 68ms/step - loss: 0.2841 - accuracy: 0.8836 - val_loss: 0.3302 - val_accuracy: 0.8552
Epoch 11/20
782/782 [==============================] - ETA: 0s - loss: 0.2785 - accuracy: 0.8843
Epoch 11: val_loss did not improve from 0.31732
782/782 [==============================] - 54s 69ms/step - loss: 0.2785 - accuracy: 0.8843 - val_loss: 0.3202 - val_accuracy: 0.8588
Epoch 12/20
781/782 [==============

In [ ]:
model_7.evaluate(test_examples, test_labels)
model_7_preds = model_7.predict(test_examples)
model_7_results = calculate_results(test_labels, tf.round(model_7_preds))
model_7_results

782/782 [==============================] - 27s 34ms/step


{'accuracy': 86.004,
 'precision': 0.8600400576064092,
 'recall': 0.86004,
 'f1_score': 0.8600399944015997}

In [ ]:
df = pd.DataFrame([model_0_results, model_1_results, model_2_results, model_3_results, model_4_results, model_5_results, model_6_results, model_7_results])

In [ ]:
df

,accuracy,precision,recall,f1_score
0,82.956,0.834304,0.82956,0.828953
1,87.100,0.871038,0.87100,0.870997
2,84.040,0.840426,0.84040,0.840397
3,85.680,0.856804,0.85680,0.856800
4,85.852,0.864823,0.85852,0.857906
5,85.268,0.854314,0.85268,0.852510
6,84.052,0.845985,0.84052,0.839888
7,86.004,0.860040,0.86004,0.860040
